In [1]:
from hamiltonian import Hamiltonian

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

# 12 qubits
lih_jw = Hamiltonian('LiH_STO3g_12qubits', 'jw')
lih_parity = Hamiltonian('LiH_STO3g_12qubits', 'parity')
lih_bk = Hamiltonian('LiH_STO3g_12qubits', 'bk')

In [3]:
ham = h2_jw

In [4]:
H = ham.SummedOp()
ground_energy, ground_state = ham.ground()

In [37]:
import numpy as np
from qiskit.aqua.operators import PauliOp, SummedOp

def generateBasis(H: SummedOp) -> str:
    n = H.num_qubits
    qubits_shift = list(np.random.choice(range(n), size=n, replace=False))
    bases_shift = []
    for j in range(n):
        basisSingle = generateBasisSingle(j, qubits_shift, bases_shift, H)
        bases_shift.append(basisSingle)
    B = '' # measurement basis
    for i in range(n):
        j = qubits_shift.index(i)
        B = B + bases_shift[j]
    return B

def generateBasisSingle(j: int, qubits_shift: list, bases_shift: list, H: SummedOp) -> str:
    assert len(bases_shift) == j
    beta = generateBeta(j, qubits_shift, bases_shift, H)
    basis = np.random.choice(['X', 'Y', 'Z'], p=beta)
    return basis

def generateBeta(j, qubits_shift, bases_shift, H):
    constants = [0.0, 0.0, 0.0]
    p2index = {'X': 0, 'Y': 1, 'Z': 2}
    for x in H:
        coeff, pauli = x.coeff, str(x.primitive)
        if isCompatible(pauli, j, qubits_shift, bases_shift):
            p = pauli[qubits_shift[j]]
            index = p2index[p]
            constants[index] += coeff**2
    beta_unnormalized = np.sqrt(constants)
    beta = beta_unnormalized / np.sum(beta_unnormalized)
    return beta

def isCompatible(pauli, j, qubits_shift, bases_shift):
    if pauli[qubits_shift[j]] == 'I':
        return False
    for k in range(j):
        i = qubits_shift[k]
        if not pauli[i] in ('I', bases_shift[k]):
            return False
    return True

def precomputePauliFrequencies(H: SummedOp, M: int) -> dict:
    out = {}
    for x in H:
        coeff, P = x.coeff, str(x.primitive)
        out[P] = 0
    for i in range(M):
        B = generateBasis(H)
        for x in H:
            coeff, P = x.coeff, str(x.primitive)
            if all([P[j]==B[j] or P[j]=='I' for j in range(len(P))]):
                out[P] += 1
    return out

def updateHamiltonian(H: SummedOp, MPs: dict, B: str) -> SummedOp:
    MPs_new = {}
    scaleFactors = []
    
    for x in H:
        coeff, P = x.coeff, str(x.primitive)
        if all([P[j]==B[j] or P[j]=='I' for j in range(len(P))]):
            MPs_new[P] = MPs[P]-1
            if MPs[P] > 1:
                scaleFactors.append((MPs[P]-1)/MPs[P])
            else:
                scaleFactors.append(0)
        else:
            MPs_new[P] = MPs[P]
            scaleFactors.append(1)
            
    return SummedOp([H[i].mul(scaleFactors[i]) for i in range(len(H))]), MPs_new

In [41]:
n_shots = 1000

MPs = precomputePauliFrequencies(H,n_shots)
print(MPs)

H_current = SummedOp([H[i] for i in range(len(H))])

for i in range(n_shots):
    B = generateBasis(H_current)
    H_current, MPs = updateHamiltonian(H_current, MPs, B)

{'YZYIIIII': 18, 'XZXIIIII': 28, 'IIIIIIII': 1000, 'ZIIIIIII': 779, 'IIZIIIII': 865, 'IZIIIIII': 855, 'IYZYIIII': 11, 'IXZXIIII': 9, 'IIIZIIII': 892, 'IIIIZIII': 789, 'IIIIYZYI': 21, 'IIIIXZXI': 29, 'IIIIIZII': 840, 'IIIIIYZY': 9, 'IIIIIXZX': 5, 'IIIIIIZI': 834, 'IIIIIIIZ': 909, 'IYIYIIII': 11, 'IXIXIIII': 9, 'XIXIIIII': 29, 'YIYIIIII': 18, 'YZYIIZII': 13, 'XZXIIZII': 23, 'XXXXIIII': 0, 'YXXYIIII': 0, 'XXYYIIII': 0, 'YYXXIIII': 0, 'XYYXIIII': 0, 'YYYYIIII': 0, 'YYIIIXXI': 0, 'YYIIIYYI': 0, 'XXIIIXXI': 0, 'XXIIIYYI': 0, 'ZIIZIIII': 701, 'YZZYIXXI': 0, 'YZZYIYYI': 0, 'XZZXIXXI': 0, 'XZZXIYYI': 0, 'ZYZYIIII': 8, 'ZXZXIIII': 7, 'ZIIIIXZX': 4, 'ZIIIIYZY': 8, 'YZYIIXZX': 1, 'YZYIIYZY': 0, 'XZXIIXZX': 0, 'XZXIIYZY': 1, 'ZZIIIIII': 662, 'ZIIIYZYI': 20, 'ZIIIXZXI': 24, 'YZYIYZYI': 0, 'YZYIXZXI': 1, 'XZXIYZYI': 0, 'XZXIXZXI': 1, 'ZIIIIIZI': 644, 'YZYIIIZI': 14, 'XZXIIIZI': 26, 'YYIIIIXX': 0, 'YYIIIIYY': 0, 'XXIIIIXX': 0, 'XXIIIIYY': 0, 'IZZIIIII': 726, 'YZZYIIXX': 0, 'YZZYIIYY': 0, 'XZZXIIXX': 0

In [42]:
print(MPs)

{'YZYIIIII': -5, 'XZXIIIII': 0, 'IIIIIIII': 0, 'ZIIIIIII': 51, 'IIZIIIII': 59, 'IZIIIIII': 44, 'IYZYIIII': -2, 'IXZXIIII': -6, 'IIIZIIII': 56, 'IIIIZIII': 52, 'IIIIYZYI': -4, 'IIIIXZXI': -1, 'IIIIIZII': 50, 'IIIIIYZY': -2, 'IIIIIXZX': -7, 'IIIIIIZI': 63, 'IIIIIIIZ': 66, 'IYIYIIII': -5, 'IXIXIIII': -7, 'XIXIIIII': -1, 'YIYIIIII': -6, 'YZYIIZII': -5, 'XZXIIZII': 1, 'XXXXIIII': -1, 'YXXYIIII': 0, 'XXYYIIII': 0, 'YYXXIIII': 0, 'XYYXIIII': 0, 'YYYYIIII': -1, 'YYIIIXXI': -1, 'YYIIIYYI': -1, 'XXIIIXXI': -1, 'XXIIIYYI': 0, 'ZIIZIIII': 80, 'YZZYIXXI': 0, 'YZZYIYYI': 0, 'XZZXIXXI': -1, 'XZZXIYYI': 0, 'ZYZYIIII': -2, 'ZXZXIIII': -4, 'ZIIIIXZX': -4, 'ZIIIIYZY': 1, 'YZYIIXZX': 1, 'YZYIIYZY': 0, 'XZXIIXZX': 0, 'XZXIIYZY': -1, 'ZZIIIIII': 83, 'ZIIIYZYI': 2, 'ZIIIXZXI': 3, 'YZYIYZYI': 0, 'YZYIXZXI': 1, 'XZXIYZYI': -1, 'XZXIXZXI': 1, 'ZIIIIIZI': 80, 'YZYIIIZI': -5, 'XZXIIIZI': 8, 'YYIIIIXX': 0, 'YYIIIIYY': -2, 'XXIIIIXX': -1, 'XXIIIIYY': 0, 'IZZIIIII': 80, 'YZZYIIXX': 0, 'YZZYIIYY': 0, 'XZZXIIXX': 0, '

In [34]:
for i in range(len(H)):
    coeff, P = H[i].coeff, str(H[i].primitive)
    if all([P[j]==B[j] or P[j]=='I' for j in range(len(P))]):
        print(P, coeff, H_new[i].coeff)

IIIIIIII 1.5253256224066558 1.5238002967842492
ZIIIIIII -0.2724180193998683 -0.27205479537400185
IIZIIIII -0.6921496164450819 -0.6913343636577496
IZIIIIII -0.4043851244705001 -0.4039088169033971
IIIZIIII -1.0346287559322498 -1.0334867815879536
IIIIZIII -0.2724180193998682 -0.27207274941330306
IIIIIZII -0.4043851244705001 -0.40390083090826
IIIIIIZI -0.6921496164450818 -0.69132168628187
IIIIIIIZ -1.0346287559322498 -1.0334791684256583
ZIIZIIII 0.13095738454560127 0.13076564606163993
ZZIIIIII 0.08831604919069061 0.0881756421172555
ZIIIIIZI 0.1328282413043635 0.1326174028261026
IZZIIIII 0.08647780044339264 0.08635531064106489
ZIIIIIIZ 0.16521846395853979 0.16497260314907766
ZIZIIIII 0.10556517226823464 0.10540301378548773
IZIIIIZI 0.09542765898647504 0.0952921083345056
IZIIIIIZ 0.11066162842599216 0.11051659352635128
IZIZIIII 0.09368490275669497 0.09356494769554684
ZIIIZIII 0.1619798167438987 0.16171029957793548
IZIIIZII 0.09652650762551396 0.09638997791175792
IZIIZIII 0.1084835363944962 0

In [8]:
MPs = precomputePauliFrequencies(H,10)
MPs

{'YZYIIIII': 0,
 'XZXIIIII': 0,
 'IIIIIIII': 10,
 'ZIIIIIII': 9,
 'IIZIIIII': 7,
 'IZIIIIII': 10,
 'IYZYIIII': 0,
 'IXZXIIII': 0,
 'IIIZIIII': 8,
 'IIIIZIII': 7,
 'IIIIYZYI': 0,
 'IIIIXZXI': 0,
 'IIIIIZII': 9,
 'IIIIIYZY': 0,
 'IIIIIXZX': 0,
 'IIIIIIZI': 10,
 'IIIIIIIZ': 9,
 'IYIYIIII': 0,
 'IXIXIIII': 0,
 'XIXIIIII': 0,
 'YIYIIIII': 0,
 'YZYIIZII': 0,
 'XZXIIZII': 0,
 'XXXXIIII': 0,
 'YXXYIIII': 0,
 'XXYYIIII': 0,
 'YYXXIIII': 0,
 'XYYXIIII': 0,
 'YYYYIIII': 0,
 'YYIIIXXI': 0,
 'YYIIIYYI': 0,
 'XXIIIXXI': 0,
 'XXIIIYYI': 0,
 'ZIIZIIII': 7,
 'YZZYIXXI': 0,
 'YZZYIYYI': 0,
 'XZZXIXXI': 0,
 'XZZXIYYI': 0,
 'ZYZYIIII': 0,
 'ZXZXIIII': 0,
 'ZIIIIXZX': 0,
 'ZIIIIYZY': 0,
 'YZYIIXZX': 0,
 'YZYIIYZY': 0,
 'XZXIIXZX': 0,
 'XZXIIYZY': 0,
 'ZZIIIIII': 9,
 'ZIIIYZYI': 0,
 'ZIIIXZXI': 0,
 'YZYIYZYI': 0,
 'YZYIXZXI': 0,
 'XZXIYZYI': 0,
 'XZXIXZXI': 0,
 'ZIIIIIZI': 9,
 'YZYIIIZI': 0,
 'XZXIIIZI': 0,
 'YYIIIIXX': 0,
 'YYIIIIYY': 0,
 'XXIIIIXX': 0,
 'XXIIIIYY': 0,
 'IZZIIIII': 7,
 'YZZYIIXX': 0,
 'YZZ

# Measure Hamiltonian in basis

In [6]:
from qiskit import QuantumCircuit, execute
from qiskit import Aer
simulator = Aer.get_backend('qasm_simulator')

In [7]:
def runAndMeasure(state, basis):
    n = len(basis)
    circ = QuantumCircuit(n, n)
    circ.initialize(state, range(n))

    circ = circ.compose(measurementCircuit(basis))

    # run experiment
    result = execute(circ, simulator, shots=1).result()
    counts = result.get_counts(circ)
    # counts is a dictionary with only one entry (since shots=1)
    bitString = counts.popitem()[0]  # physics ordering
    
    # return +/- evalues
    evalues = [(-1)**int(bit) for bit in bitString]
    return evalues

def measurementCircuit(basis: str):
    n = len(basis)
    circ = QuantumCircuit(n, n)
    # qiskit ordering
    for qubit, pauli in enumerate(basis[::-1]):
        circ = measurementPauli(circ, pauli, qubit)
    return circ


def measurementPauli(circ, pauli, qubit):
    '''
    modify circuit by appending measurement.
    return modified circuit
    '''
    if pauli == 'X':
        circ.h(qubit)
    elif pauli == 'Y':
        circ.sdg(qubit)
        circ.h(qubit)
    elif pauli == 'Z':
        pass
    circ.measure(qubit, qubit)
    return circ

In [8]:
evalues = runAndMeasure(ground_state, basis)
evalues

[-1, -1, 1, 1, 1, 1, 1, 1]

# Accumulate data

In [9]:
def buildPauliEstimates(H):
    pauliEstimates = {}
    # key = Pauli appearing in H
    # value = dict where
        # number = number of times a basis has allowed Pauli to be estimated
        # running = list of running best estimates of Pauli value
    for x in H:
        pauli = str(x.primitive)
        pauliEstimates[pauli] = {'number': 0, 'running': [0.0]}
    return pauliEstimates
        
def isEstimatible(pauli, basis):
    for qubit in range(len(basis)):
        if not pauli[qubit] in ('I', basis[qubit]):
            return False
    return True

def estimate(pauli, evalues):
    est = 1.0
    for qubit, p in enumerate(pauli):
        if p != 'I':
            est *= evalues[qubit]
    return est

def updatePauliEstimates(pauliEstimates, evalues, basis):
    for x in H:
        pauli = str(x.primitive)
        lastEstimate = pauliEstimates[pauli]['running'][-1]
        if isEstimatible(pauli, basis):
            est = estimate(pauli, evalues)
            n = pauliEstimates[pauli]['number']
            newEstimate = 1/(n+1) * (n * lastEstimate + est)
            pauliEstimates[pauli]['number'] += 1
            pauliEstimates[pauli]['running'].append(newEstimate)
        else:
            pauliEstimates[pauli]['running'].append(lastEstimate)
    pass

In [10]:
#def energyEstimates(H, pauliEstimates):
#    energies = [0.0]
#    for x in H:
#        coeff, pauli = x.coeff, str(x.primitive)
#        energy += 0

# Simulation

In [11]:
ham = h2_jw
H = ham.SummedOp()

In [12]:
%%time
ground_energy, ground_state = ham.ground(sparse=True)

CPU times: user 139 ms, sys: 11.1 ms, total: 150 ms
Wall time: 108 ms


In [13]:
pauliEstimates = buildPauliEstimates(H)

In [14]:
n_shots = 1000

In [15]:
%%time
for shot in range(n_shots):
    basis = generateBasis(H)
    evalues = runAndMeasure(ground_state, basis)
    updatePauliEstimates(pauliEstimates, evalues, basis)

CPU times: user 22.6 s, sys: 1.63 s, total: 24.2 s
Wall time: 24 s


In [16]:
energyEstimates = [0.0]
for shot in range(n_shots):
    energyRunning = 0.0
    for x in H:
        coeff, pauli = x.coeff, str(x.primitive)
        energyRunning += coeff * pauliEstimates[pauli]['running'][shot+1]
    energyEstimates.append(energyRunning)

In [17]:
print('true       :', ground_energy)
print('estimate   :', energyEstimates[-1])
print('difference :', energyEstimates[-1] - ground_energy)

true       : -1.8608605555207562
estimate   : -1.8138562412496944
difference : 0.04700431427106189


# TEST

In [18]:
from qiskit.quantum_info import Pauli
from qiskit.opflow import PauliOp, SummedOp

In [19]:
def testSummedOp(dictionary):
    paulis = []
    for P, coeff_P in dictionary.items():
        paulis.append(coeff_P * PauliOp(Pauli.from_label(P)))
    return SummedOp(paulis)

def testGround(SummedOp):
    mat = SummedOp.to_matrix()
    evalues, evectors = np.linalg.eigh(mat)
    index = np.argmin(evalues)
    ground_energy = evalues[index]
    ground_state = evectors[:,index]
    return ground_energy, ground_state

In [20]:
dictionary = {'ZII': -1, 'IZI': -1, 'IIX': 1}

H = testSummedOp(dictionary)
ground_energy, ground_state = testGround(H)

In [21]:
basis = generateBasis(H)
#print(basis)

In [22]:
n = 3
circ = QuantumCircuit(n, n)
circ.initialize(ground_state, range(n))
circ.draw()

┌───────────────────────────────────────────┐
q_0: ┤0                                          ├
     │                                           │
q_1: ┤1 initialize(0.70711,-0.70711,0,0,0,0,0,0) ├
     │                                           │
q_2: ┤2                                          ├
     └───────────────────────────────────────────┘
c: 3/═════════════════════════════════════════════

In [23]:
measurementCircuit(basis).draw()

┌───┐   ┌─┐
q_0: ┤ H ├───┤M├
     └┬─┬┘   └╥┘
q_1: ─┤M├─────╫─
      └╥┘ ┌─┐ ║ 
q_2: ──╫──┤M├─╫─
       ║  └╥┘ ║ 
c: 3/══╩═══╩══╩═
       1   2  0

In [24]:
circ = circ.compose(measurementCircuit(basis))
circ.draw()

┌───────────────────────────────────────────┐┌───┐   ┌─┐
q_0: ┤0                                          ├┤ H ├───┤M├
     │                                           │└┬─┬┘   └╥┘
q_1: ┤1 initialize(0.70711,-0.70711,0,0,0,0,0,0) ├─┤M├─────╫─
     │                                           │ └╥┘ ┌─┐ ║ 
q_2: ┤2                                          ├──╫──┤M├─╫─
     └───────────────────────────────────────────┘  ║  └╥┘ ║ 
c: 3/═══════════════════════════════════════════════╩═══╩══╩═
                                                    1   2  0

In [25]:
result = execute(circ, simulator, shots=1).result()
counts = result.get_counts(circ)
# counts is a dictionary with only one entry (since shots=1)
bitString = counts.popitem()[0]
#bitString